In [1]:
import torch
torch.__version__

'1.7.1'

In [2]:
torch.cuda.is_available()

True

In [3]:
device_cnt = torch.cuda.device_count()
cur_device = torch.cuda.current_device()
device_name = torch.cuda.get_device_name(cur_device)

print(f"Number of CUDA-enabled GPUs: {device_cnt}\nCurrent Device ID: {cur_device}\nCurrent Device Name: {device_name}")

Number of CUDA-enabled GPUs: 1
Current Device ID: 0
Current Device Name: GeForce GTX 1650


Data is downloaded from: https://www.kaggle.com/borismarjanovic/price-volume-data-for-all-us-stocks-etfs

(just using the Stocks data, not ETF)

In [4]:
from fastai.data.all import file_extract
from pathlib import Path

if not Path('./Stocks').exists():
    file_extract('./Stocks.zip', '.')
else:
    print('Stocks data exists')

Stocks data exists


In [5]:
from fastai.data.all import *

In [6]:
files = get_files('./Stocks')

In [7]:
some_files = files[::100]
some_files

(#72) [Path('Stocks/a.us.txt'),Path('Stocks/adro.us.txt'),Path('Stocks/ahpa.us.txt'),Path('Stocks/amag.us.txt'),Path('Stocks/apf.us.txt'),Path('Stocks/asnd.us.txt'),Path('Stocks/axs_e.us.txt'),Path('Stocks/bctf.us.txt'),Path('Stocks/bkj.us.txt'),Path('Stocks/bpt.us.txt')...]

In [8]:
# benchmark these

# brute-force
# def categorize_growth1(filepath):
#     with filepath.open() as f:
#         f.readline()
#         first = last = f.readline()
#         for cont in f:
#             last = f.readline()
#         return first, last
        
# read-back byte by byte, then readline when encountering a newline
def categorize_growth2(filepath):
    with filepath.open('rb+') as f:
        f.readline()
        first = f.readline().strip(b'\r\n')
        f.seek(-2, 2)
        while f.read(1) != b'\n': f.seek(-2, 1)
        return first, f.readline().strip(b'\r\n')

# same as approach , but build last line as you read(probably have to reverse + join)
# def categorize_growth3(filepath):
#     with filepath.open('rb+') as f:
#         f.readline()
#         first = f.readline().strip(b'\r\n')
#         f.seek(-2, 2)
#         last_line = [f.read(1)]
#         while last_line[-1] != b'\n':
#             f.seek(-2, 1)
#             last_line.append(f.read(1))
#         return first, b''.join(last_line)[::-1].strip(b'\r\n')

# read-back byte by byte, then readline when encountering a newline (read more to be more efficient)
# Potential issue: assumes to overread once
def categorize_growth4(filepath):
    with filepath.open('rb+') as f:
        f.readline()
        first = f.readline().strip(b'\r\n')
        f.seek(-11, 2)
        while b'\n' not in f.read(10): f.seek(-20, 1)
        f.seek(-10, 1)
        f.readline()
        return first, f.readline().strip(b'\r\n')

In [9]:
# [categorize_growth2(filepath) for filepath in some_files]
import os
def nonempty_file(filepath): return os.path.getsize(filepath) != 0
    
map2 = dict()
for filepath in filter(nonempty_file, some_files):
    map2[filepath] = categorize_growth2(filepath)

In [10]:
map4 = dict()
for filepath in filter(nonempty_file, some_files):
    map4[filepath] = categorize_growth4(filepath)

In [11]:
if len(map2) != len(map4): print('UNEQUAL')
for k,v in map2.items():
    if map4[k] == v:
        print('[PASS]', k)
    else: 
        print('[FAIL]', v, '- got', map4[k])

[PASS] Stocks\a.us.txt
[PASS] Stocks\adro.us.txt
[PASS] Stocks\ahpa.us.txt
[PASS] Stocks\amag.us.txt
[PASS] Stocks\apf.us.txt
[PASS] Stocks\asnd.us.txt
[PASS] Stocks\axs_e.us.txt
[PASS] Stocks\bctf.us.txt
[PASS] Stocks\bkj.us.txt
[PASS] Stocks\bpt.us.txt
[PASS] Stocks\bxmx.us.txt
[PASS] Stocks\cbs.us.txt
[PASS] Stocks\cffi.us.txt
[PASS] Stocks\civi.us.txt
[PASS] Stocks\cncr.us.txt
[PASS] Stocks\cpt.us.txt
[PASS] Stocks\ctw.us.txt
[PASS] Stocks\c_l.us.txt
[PASS] Stocks\dhr.us.txt
[PASS] Stocks\dspg.us.txt
[PASS] Stocks\ebtc.us.txt
[PASS] Stocks\eltk.us.txt
[PASS] Stocks\erii.us.txt
[PASS] Stocks\ewmc.us.txt
[PASS] Stocks\fet.us.txt
[PASS] Stocks\fmcir.us.txt
[PASS] Stocks\fss.us.txt
[PASS] Stocks\gdp.us.txt
[PASS] Stocks\glow.us.txt
[PASS] Stocks\grp-u.us.txt
[PASS] Stocks\hayn.us.txt
[PASS] Stocks\hii.us.txt
[PASS] Stocks\htd.us.txt
[PASS] Stocks\icsh.us.txt
[PASS] Stocks\ingn.us.txt
[PASS] Stocks\itek.us.txt
[PASS] Stocks\jmom.us.txt
[PASS] Stocks\ked.us.txt
[PASS] Stocks\kt.us.txt
[P

In [12]:
# benchmark
import timeit
# print(timeit.timeit("categorize_growth1(some_files[0])", globals=locals(), number=10000)) ~ 10 s
print(timeit.timeit("categorize_growth2(some_files[0])", globals=locals(), number=10000))
# print(timeit.timeit("categorize_growth3(some_files[0])", globals=locals(), number=10000)) ~ 2.841 s, very similar to approach 2
print(timeit.timeit("categorize_growth4(some_files[0])", globals=locals(), number=10000))

3.032823200000001
0.9754511000000008


In [13]:
DataBlock(
#     blocks = (, CategoryBlock),
    get_items = get_files,
    get_y = categorize_growth4,
#     item_tfms = []
    splitter = RandomSplitter(valid_pct=0.2, seed=42)
)

NameError: name 'categorize_growth1' is not defined